In [26]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime

In [40]:
tra = pd.read_csv('/Users/suzukishinji/kaggle/recluit/air_visit_data.csv')
tes = pd.read_csv('/Users/suzukishinji/kaggle/recluit/sample_submission-3.csv')

In [41]:
#あとでデータ数を確認するため
print(tra.shape)
print(tes.shape)
print('↑元データ')

(252108, 3)
(32019, 2)
↑元データ


In [49]:
from dateutil.parser import parse
from datetime import date, timedelta

start_date = '2017-03-12'
end_date = parse(start_date[:10]) + timedelta(days=days)
end_date = end_date.strftime('%Y-%m-%d')
days = (parse(day1[:10]) - parse(day2[:10])).days

temp = tra[(tra.visit_date < key[0]) & (tra.visit_date > start_date)]

NameError: name 'days' is not defined

In [43]:
temp.head()

,air_store_id,visit_date,visitors
353,air_ba937bf13d40fb24,2017-03-13,7
354,air_ba937bf13d40fb24,2017-03-14,7
355,air_ba937bf13d40fb24,2017-03-15,21
356,air_ba937bf13d40fb24,2017-03-16,12
357,air_ba937bf13d40fb24,2017-03-17,35


In [38]:
tra['visit_date'] = pd.to_datetime(tra['visit_date'])
tra['visit_date'] = pd.to_datetime(tra['visit_date'])
tra['dow'] = tra['visit_date'].dt.dayofweek
tra['year'] = tra['visit_date'].dt.year
tra['month'] = tra['visit_date'].dt.month
tra['visit_date'] = tra['visit_date'].dt.date

In [39]:
start_date = '2017-03-12'
temp = tra[(tra.visit_date > start_date )].head()

TypeError: '>' not supported between instances of 'datetime.date' and 'str'

In [22]:
tes['visit_date'] = tes['id'].map(lambda x: str(x).split('_')[2])
tes['air_store_id'] = tes['id'].map(lambda x: '_'.join(x.split('_')[:2]))
tes['visit_date'] = pd.to_datetime(tes['visit_date'])
tes['dow'] = tes['visit_date'].dt.dayofweek
tes['year'] = tes['visit_date'].dt.year
tes['month'] = tes['visit_date'].dt.month
tes['visit_date'] = tes['visit_date'].dt.date

In [23]:
#リョウさんのGW_flagを利用させて頂きました（ここの日付の範囲は適当に3週間としたので変更しても良いと思います）
combine = [tra, tes]
gw_list = ['2016-04-23','2016-04-24','2016-04-25','2016-04-26','2016-04-27','2016-04-28','2016-04-29','2016-04-30','2016-04-23','2016-05-01','2016-05-02','2016-05-03','2016-05-04','2016-05-05','2016-05-06','2016-05-07','2016-05-08','2016-05-09','2016-05-10','2016-05-11','2016-05-12','2016-05-13','2016-05-14','2016-05-15','2017-04-22','2017-04-23','2017-04-24','2017-04-25','2017-04-26','2017-04-27','2017-04-28','2017-04-29','2017-04-30','2017-05-01','2017-05-02','2017-05-03','2017-05-04','2017-05-05','2017-05-06','2017-05-07','2017-05-08','2017-05-09','2017-05-10','2017-05-11','2017-05-12','2017-05-13','2017-05-14']
tra['gw_flg'] = 0
tes['gw_flg'] = 0
update_gw_list = [["0" for i in range(3)] for j in range(len(gw_list))]

from datetime import date
for index, gw_date in enumerate(gw_list):
    temp_list = gw_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_gw_list[index][col_i]=int(temp_figure)
        
    #print("{}  {}  {}".format(update_list[index][0],update_list[index][1],update_list[index][2]))

for dataset in combine:
    for index in range(len(update_gw_list)):
        dataset.loc[dataset.visit_date == date(update_gw_list[index][0],update_gw_list[index][1],update_gw_list[index][2]), 'gw_flg'] = 1

#gw部分とそれ以外を分けます。今回はtraはgwを含んで学習する
tra_gw = tra[tra['gw_flg'] == 1]
#tra = tra[tra['gw_flg'] == 0]

tes_gw = tes[tes['gw_flg'] == 1]
#tes = tes[tes['gw_flg'] == 0]

In [24]:
#513店舗ように316店舗の2016/4/23〜2016/5/31（テスト期間だけ）のtrainデーたを作る

combine = [tra, tes]
test_list = ['2016-04-23','2016-04-24','2016-04-25','2016-04-26','2016-04-27','2016-04-28','2016-04-29','2016-04-30',
           '2016-04-23','2016-05-01','2016-05-02','2016-05-03','2016-05-04','2016-05-05','2016-05-06','2016-05-07',
           '2016-05-08','2016-05-09','2016-05-10','2016-05-11','2016-05-12','2016-05-13','2016-05-14','2016-05-15',
          '2016-05-16','2016-05-17','2016-05-18','2016-05-19','2016-05-20','2016-05-21','2016-05-22','2016-05-23','2016-05-24'
          ,'2016-05-25','2016-05-26','2016-05-27','2016-05-28','2016-05-29','2016-05-30','2016-05-31']
tra['test_flg'] = 0
update_test_list = [["0" for i in range(3)] for j in range(len(test_list))]

from datetime import date
for index, test_date in enumerate(test_list):
    temp_list = test_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_test_list[index][col_i]=int(temp_figure)
        
    #print("{}  {}  {}".format(update_list[index][0],update_list[index][1],update_list[index][2]))

for dataset in combine:
    for index in range(len(update_test_list)):
        dataset.loc[dataset.visit_date == date(update_test_list[index][0],update_test_list[index][1],update_test_list[index][2]), 'test_flg'] = 1

#test部分とそれ以外を分けます。今回はtraはgwを含んで学習する
tra_test = tra[tra['test_flg'] == 1]
#tra = tra[tra['gw_flg'] == 0]

#tes_gw = tes[tes['gw_flg'] == 1]
#tes = tes[tes['gw_flg'] == 0]

In [25]:
tra_test.shape

(10336, 8)

In [12]:
#2016/1~2016/6に営業しているお店は316店舗でした。
year_2016 = tra[tra['year'] == 2016]
month_1 = year_2016[year_2016['month'] == 1]
month_2 = year_2016[year_2016['month'] == 2]
month_3 = year_2016[year_2016['month'] == 3]
month_4 = year_2016[year_2016['month'] == 4]
month_5 = year_2016[year_2016['month'] == 5]
month_6 = year_2016[year_2016['month'] == 6]
tra_store_316_6 = pd.concat([month_1,month_2,month_3,month_4,month_5,month_6])

In [13]:
id = list(tra_store_316_6['air_store_id'].values.flatten())

#trainデータから2016/1~2016/6に営業しているお店の全期間を抜き出したもの。
#trainデータから上記を除いたものが513店舗。
tra_store_316_all = tra[tra['air_store_id'].isin(id)]
tra_store_513 = tra[~tra['air_store_id'].isin(id)]

#testデータから2016/1~2016/6に営業しているお店の全期間を抜き出したもの。
#testデータから上記を除いたものが513店舗。
tes_store_316 = tes[tes['air_store_id'].isin(id)]
tes_store_513 = tes[~tes['air_store_id'].isin(id)]

In [15]:
tra_513_tes = pd.concat([tra_store_513, tra_test])

In [16]:

#使うのは①2016/1~2016/6の期間のデータ、②2016/1~2016/6に営業していたお店の全期間のデータ、③途中からの513店舗のデータ。
#tra_store_316_6[['air_store_id', 'visit_date','visitors']].to_csv('tra_store_316_6.csv', index = False)
tra_513_tes[['air_store_id', 'visit_date','visitors']].to_csv('tra_store_513_tes.csv', index = False)
tra_store_316_all[['air_store_id', 'visit_date','visitors']].to_csv('tra_store_316_all.csv', index = False)
#tra_store_513[['air_store_id', 'visit_date','visitors']].to_csv('tra_store_513.csv', index = False)
tra_gw[['air_store_id', 'visit_date','visitors']].to_csv('tra_gw.csv', index = False)

#テストデータは期間で分けずに最初からの316店舗と途中からの513店舗
tes_store_316[['id','visitors']].to_csv('tes_store_316.csv', index = False)
tes_store_513[['id','visitors']].to_csv('tes_store_513.csv', index = False)
tes_gw[['id','visitors']].to_csv('tes_gw.csv', index = False)

In [17]:
tra_316_all = pd.read_csv('/Users/suzukishinji/kaggle/recluit/tra_store_316_all.csv')
tra_513_tes = pd.read_csv('/Users/suzukishinji/kaggle/recluit/tra_store_513_tes.csv')
tra_gw = pd.read_csv('/Users/suzukishinji/kaggle/recluit/tra_gw.csv')

tes_316 = pd.read_csv('/Users/suzukishinji/kaggle/recluit/tes_store_316.csv')
tes_513 = pd.read_csv('/Users/suzukishinji/kaggle/recluit/tes_store_513.csv')
tes_gw = pd.read_csv('/Users/suzukishinji/kaggle/recluit/tes_gw.csv')

#出力されているか確認。（①と③はくっつけて使う）
print(tra_316_all.shape)
print(tra_513_tes.shape)
print(tra_gw.shape)
print()
print()
print(tes_316.shape)
print(tes_513.shape)
print(tes_gw.shape)

(126700, 3)
(135744, 3)
(6793, 3)


(12207, 2)
(19812, 2)
(18062, 2)
